# Testing performance of sklearn_flatten and sklearn_unflatten vs raster_stack and raster_unstack

In [1]:
# Load the necessary Python packages.
%matplotlib inline

import warnings
import datacube
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolours
from matplotlib.patches import Patch

warnings.filterwarnings("ignore")

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)

from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import display_map, plot_lulc
from deafrica_tools.classification import sklearn_flatten, sklearn_unflatten
from deafrica_tools.dask import create_local_dask_cluster

In [2]:
dc = datacube.Datacube(app="Test")

In [3]:
# Define the area of interest.
central_lat = -1.2933
central_lon =  36.8379

lat_buffer = 0.1
lon_buffer = 0.1

# Combine lat, lon with their respective buffers to get area of interest.
lat_range = (central_lat - lat_buffer, central_lat + lat_buffer)
lon_range = (central_lon - lon_buffer, central_lon + lon_buffer)

# Time frame for the analysis.
time_range = ("2020-01", "2020-04")

# View the study area
display_map(x=lon_range, y=lat_range)

In [4]:
## Load Sentinel 1 data
query = {
    "y": lat_range,
    "x": lon_range,
    "time": time_range,
    "output_crs": "EPSG:6933",
    "resolution": (-20, 20),
}

ds = load_ard(
    dc=dc, products=["s1_rtc"], measurements=["vv", "vh"], group_by="solar_day", **query
)

print(ds)

Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 30 time steps


CPLReleaseMutex: Error = 1 (Operation not permitted)


<xarray.Dataset>
Dimensions:      (time: 30, y: 1276, x: 965)
Coordinates:
  * time         (time) datetime64[ns] 2020-01-04T15:56:20.113636 ... 2020-04...
  * y            (y) float64 -1.522e+05 -1.522e+05 ... -1.777e+05 -1.777e+05
  * x            (x) float64 3.545e+06 3.545e+06 ... 3.564e+06 3.564e+06
    spatial_ref  int32 6933
Data variables:
    vv           (time, y, x) float32 0.2355 0.1498 0.08076 ... 0.2787 0.2009
    vh           (time, y, x) float32 0.05043 0.04586 ... 0.08394 0.06705
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref


## Test 1 : Performance of sklearn_flatten, raster_stack and raster_stack2 on a dataset with NaNs

The following test shows the difference between the output of raster_stack , raster_stack2 and sklearn_flatten. 
Unlike sklearn_flatten and raster_stack2, raster_stack does not drop the NaN's but fills them with an arbitrary value `-9999`.
This results in the difference in the shape and values of the raster_stack and sklearn_flatten and raster_stack2 values. 
The raster_stack function is the fastest.

In [5]:
def raster_stack(input_xr):
    """
    Reshape a DataArray or Dataset with spatial (and optionally
    temporal) structure into an np.array with the spatial and temporal
    dimensions flattened into one dimension.
    This flattening procedure enables DataArrays and Datasets to be used
    to train and predict with sklearn models.

    Last modified: December 2021

    Parameters
    ----------
    input_xr : xarray.DataArray or xarray.Dataset
        Must have dimensions 'x' and 'y', may have dimension 'time'.
        Dimensions other than 'x', 'y' and 'time' are unaffected by the
        flattening.
    Returns
    ----------
    input_np : numpy.array
        A numpy array corresponding to input_xr.data (or
        input_xr.to_array().data), with dimensions 'x','y' and 'time'
        flattened into a single dimension, which is the first axis of
        the returned array. input_np contains no NaNs.
    """

    # Cast input Datasets to DataArray.
    if isinstance(input_xr, xr.Dataset):
        input_xr = input_xr.to_array()
        
    # Fill the Nan's in the dataset.
    # input_xr = input_xr.fillna(-9999)
  
    # Get the input_xr's data as a numpy.ndarray.
    input_xr_npa = input_xr.values
    
    # Define function to flatten and drop NaNs from a numpy array. 
    def flatten_drop_nan(np_array):
        np_flattened = np_array.flatten(order="F")
        np_drop_nan = np_flattened[np.logical_not(np.isnan(np_flattened))]
        return np_drop_nan
   
    # Flatten the numpy array.
    input_xr_flattened_npa = np.array([flatten_drop_nan(input_xr_npa[i]) for i in range(input_xr_npa.shape[0])])
  
    # Transpose the flattened numpy array to get the shape (n_samples, n_features)
    # where n_features is the number of bands and n_samples is the total number of pixels in each band.
    input_np = input_xr_flattened_npa.T

    return input_np

In [6]:
# The ds dataset contains NaNs.
print(ds)

<xarray.Dataset>
Dimensions:      (time: 30, y: 1276, x: 965)
Coordinates:
  * time         (time) datetime64[ns] 2020-01-04T15:56:20.113636 ... 2020-04...
  * y            (y) float64 -1.522e+05 -1.522e+05 ... -1.777e+05 -1.777e+05
  * x            (x) float64 3.545e+06 3.545e+06 ... 3.564e+06 3.564e+06
    spatial_ref  int32 6933
Data variables:
    vv           (time, y, x) float32 0.2355 0.1498 0.08076 ... 0.2787 0.2009
    vh           (time, y, x) float32 0.05043 0.04586 ... 0.08394 0.06705
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref


In [7]:
# Count the number of NaNs in the ds Dataset.
np.count_nonzero(np.isnan(ds.to_array().values))

1781184

In [8]:
%time sklearn_flatten(ds)

CPU times: user 1.42 s, sys: 573 ms, total: 1.99 s
Wall time: 1.99 s


array([[0.23554994, 0.0504317 ],
       [0.13722907, 0.05083345],
       [0.4322208 , 0.13801377],
       ...,
       [0.19534045, 0.08038338],
       [0.31594756, 0.07598579],
       [0.20093839, 0.067047  ]], dtype=float32)

In [9]:
%time raster_stack(ds)

CPU times: user 493 ms, sys: 264 ms, total: 757 ms
Wall time: 755 ms


array([[0.23554994, 0.0504317 ],
       [0.13722907, 0.05083345],
       [0.4322208 , 0.13801377],
       ...,
       [0.19534045, 0.08038338],
       [0.31594756, 0.07598579],
       [0.20093839, 0.067047  ]], dtype=float32)

In [10]:
X = sklearn_flatten(ds)
X1 = raster_stack(ds)
print(X.shape)
print(X1.shape)

(36049608, 2)
(36049608, 2)


In [11]:
# Evaluate if the output of sklearn_flatten and raster_stack are equal.
np.unique(X == X1)

array([ True])

## Test 2 : Performance of sklearn_flatten, raster_stack and raster_stack2 on a dataset without NaNs

On a dataset with no NaNS the sklearn_flatten, raster_stack and raster_stack2 functions all have the same output.
The raster_stack function is the fastest.

In [12]:
# The ds_median Dataset does not have any NaNs.
ds_median = ds.median(dim="time")
print(ds_median)

<xarray.Dataset>
Dimensions:      (y: 1276, x: 965)
Coordinates:
  * y            (y) float64 -1.522e+05 -1.522e+05 ... -1.777e+05 -1.777e+05
  * x            (x) float64 3.545e+06 3.545e+06 ... 3.564e+06 3.564e+06
    spatial_ref  int32 6933
Data variables:
    vv           (y, x) float32 0.1607 0.178 0.1652 ... 0.1377 0.171 0.1869
    vh           (y, x) float32 0.05063 0.04961 0.03781 ... 0.03357 0.04337


In [13]:
# Count the number of NaNs in the ds_median Dataset.
np.count_nonzero(np.isnan(ds_median.to_array().values))

0

In [14]:
%time sklearn_flatten(ds_median)

CPU times: user 39.4 ms, sys: 14.5 ms, total: 53.9 ms
Wall time: 53.3 ms


array([[0.16067713, 0.05063258],
       [0.181238  , 0.04633285],
       [0.17403632, 0.0490597 ],
       ...,
       [0.3799113 , 0.07505855],
       [0.3147898 , 0.05331162],
       [0.18691418, 0.04337489]], dtype=float32)

In [15]:
%time raster_stack(ds_median)

CPU times: user 15.8 ms, sys: 0 ns, total: 15.8 ms
Wall time: 13.8 ms


array([[0.16067713, 0.05063258],
       [0.181238  , 0.04633285],
       [0.17403632, 0.0490597 ],
       ...,
       [0.3799113 , 0.07505855],
       [0.3147898 , 0.05331162],
       [0.18691418, 0.04337489]], dtype=float32)

In [16]:
X = sklearn_flatten(ds_median)
X1 = raster_stack(ds_median)
print(X.shape)
print(X1.shape)

(1231340, 2)
(1231340, 2)


In [17]:
# Evaluate if the output of sklearn_flatten and raster_stack are equal.
np.unique(X == X1)

array([ True])

## Test 3 : Performance of sklearn_unflatten, raster_unstack on a dataset with NaNs.

In [18]:
from datacube.utils.geometry import assign_crs

def raster_unstack(output_np, input_xr):
    """
    Reshape a numpy array with no 'missing' elements (NaNs) and
    'flattened' spatiotemporal structure into a DataArray matching the
    spatiotemporal structure of the DataArray
    This enables an sklearn model's prediction to be remapped to the
    correct pixels in the input DataArray or Dataset.

    Last modified: November 2021

    Parameters
    ----------
    output_np : numpy.array
        The first dimension's length should correspond to the number of
        pixels in input_xr.

    input_xr : xarray.DataArray or xarray.Dataset
        Must have dimensions 'x' and 'y', may have dimension 'time'.
        Dimensions other than 'x', 'y' and 'time' are unaffected by the
        flattening.

    Returns
    ----------
    output_xr : xarray.DataArray
        An xarray.DataArray with the same dimensions 'x', 'y' and 'time'
        as input_xr, and the same number of pixels. The pixels
        are set to match the data in output_np.
    """

    # The  expected output of a sklearn model prediction should just be
    # a 1 dimensional numpy array, output_np, with the size/columns matching
    # the y * x * time  for the dimensions of the input_xr DataArray/Dataset.

    # Cast input Datasets to DataArray.
    if isinstance(input_xr, xr.Dataset):
        input_xr = input_xr.to_array()

    # Work around for input_xr Dataset with geographic coordinate reference system.
    if input_xr.geobox.crs.geographic:
        input_xr = input_xr.rename({"longitude": "x", "latitude": "y"})

    # Get the input_xr's data as a numpy.ndarray.
    input_xr_npa = input_xr.values
    
    # Get the data type of the input_xr_npa numpy array.
    data_type = input_xr_npa.dtype

    # Flatten the input_xr_npa numpy array.
    input_xr_flattened_npa = np.array([input_xr_npa[i].flatten(order="F") for i in range(input_xr_npa.shape[0])])

    # Transpose the input_xr_flattened_npa numpy array to get the shape (n_samples, n_features)
    # where n_features is the number of bands and n_samples is the total number of pixels in each band.
    input_np = input_xr_flattened_npa.T

    # Create a mask of the Nans.
    mask = np.isnan(input_np)

    # Create an empty numpy array..
    output_xr_np = np.empty(input_np.shape, dtype=data_type)

    # Use the mask to put the data in all the right places. 
    for i in range(output_xr_np.shape[1]):
        arr = output_xr_np[:,i]
        mask_arr = mask[:,i]
        arr[~mask_arr] = output_np[:,i]
        arr[mask_arr] = np.nan

    # Reshape the output_np numpy array.
    output_xr_np = output_xr_np.T.reshape(input_xr_npa.shape, order="F")

    # Get the dimensions for output_xr.
    if "time" in input_xr.dims:
        dims = ["variable", "time", "y", "x"]
        coords = dict(
            time=(["time"], input_xr.coords["time"].values),
            y=(["y"], input_xr.coords["y"].values),
            x=(["x"], input_xr.coords["x"].values),
            spatial_ref=input_xr.coords["spatial_ref"].values,
            variable=(["variable"], [f"band_{i}" for i in range(output_xr_np.shape[0])]),
        )
    else:
        dims = ["variable", "y", "x"]
        coords = dict(
            y=(["y"], input_xr.coords["y"].values),
            x=(["x"], input_xr.coords["x"].values),
            spatial_ref=input_xr.coords["spatial_ref"].values,
            variable=(["variable"], [f"band_{i}" for i in range(output_xr_np.shape[0])]),
        )

    # Convert the output_np numpy array into a xarray DataArray.
    output_xr = xr.DataArray(
        data=output_xr_np,
        dims=dims,
        coords=coords,
        attrs=input_xr.attrs,
    )
    # Assign the output
    output_xr = assign_crs(output_xr, input_xr.geobox.crs)
    
    # Work around for input_xr Dataset with geographic coordinate reference system.
    if output_xr.geobox.crs.geographic:
        output_xr= output_xr.rename({"x": "longitude" , "y": "latitude"})
    return output_xr

In [19]:
ds_array = ds.to_array()

In [20]:
# Flatten the DataArray.
X = sklearn_flatten(ds_array)
X1 = raster_stack(ds_array)

# Unflatten the resulting numpy arrays. 
Y = sklearn_unflatten(X, ds_array)
Y1 = raster_unstack(X1, ds_array)

The sklearn_flatten returns a DataArray in the dimension order of **variable, x, y and time**. 
Which is different to how the the DE Africa Datasets or DataArrays are normally presented which is in the dimension order **variable, time, y and x**. 

The raster_unstack returns a DataArray in the dimension order **variable, time, y and x**.

In [21]:
print(ds_array.dims)
ds_array.shape

('variable', 'time', 'y', 'x')


(2, 30, 1276, 965)

In [22]:
print(Y.dims)
Y.shape

('output_dim_0', 'x', 'y', 'time')


(2, 965, 1276, 30)

In [23]:
print(Y1.dims)
Y1.shape

('variable', 'time', 'y', 'x')


(2, 30, 1276, 965)

For the purpose of comparison I have transposed the values of the results of the sklearn_unflatten into the dimension order variable, time, y and x.

In [24]:
Y_t = np.transpose(Y.values, (0,3,2,1))
Y_t.shape

(2, 30, 1276, 965)

When transposed, the values of the sklearn_unflatten are not equal to the original DataArray which is odd because when used on a DataArray that has no NaNs (like in Test 4 below) the two are equal. 

The same thing happens in the raster_unstack.

In [25]:
# Test if the results of the transposed sklearn_unflatten and the ds_array dataArray are equal. 
np.unique(ds_array.values == Y_t) 

array([False,  True])

In [26]:
# Test if the results of the raster_unstack and the ds_array DataArray are equal. 
np.unique(ds_array.values == Y1.values) 

array([False,  True])

In [27]:
# Test if the results of raster_unstack and the results of the transposed sklearn_unflatten are equal.
np.unique(Y_t == Y1.values) 

array([False,  True])

Time the sklearn_unflatten and the raster_unstack functions. 

In [28]:
%time Y = sklearn_unflatten(X, ds_array)

CPU times: user 2min 1s, sys: 5.4 s, total: 2min 6s
Wall time: 2min 6s


In [29]:
%time Y1 = raster_unstack(X1, ds_array)

CPU times: user 506 ms, sys: 192 ms, total: 698 ms
Wall time: 696 ms


## Test 4: Performance of sklearn_flatten, raster_unstack on a dataset without NaNs.

In [30]:
# Convert the ds_median Dataset to a DataArray.
ds_median_array = ds_median.to_array()

In [31]:
# Count the number of Nans in the ds_median_array DataArray. 
np.count_nonzero(np.isnan(ds_median_array.values))

0

In [32]:
# Flatten the DataArray.
X = sklearn_flatten(ds_median_array)
X1 = raster_stack(ds_median_array)

# Unflatten the resulting numpy arrays. 
Y = sklearn_unflatten(X, ds_median_array)
Y1 = raster_unstack(X1, ds_median_array)

The sklearn_flatten returns a DataArray in the dimension order of **variable, x, y and time**. 
Which is different to how the the DE Africa Datasets or DataArrays are normally presented which is in the dimension order **variable, time, y and x**. 

The raster_unstack returns a DataArray in the dimension order **variable, time, y and x**.

In [33]:
print(ds_median_array.dims)
ds_median_array.shape

('variable', 'y', 'x')


(2, 1276, 965)

In [34]:
print(Y.dims)
Y.shape

('output_dim_0', 'x', 'y')


(2, 965, 1276)

In [35]:
print(Y1.dims)
Y1.shape

('variable', 'y', 'x')


(2, 1276, 965)

For the purpose of comparison I have transposed the values of the results of the sklearn_unflatten into the dimension order variable, time, y and x.

In [36]:
Y_t = np.transpose(Y.values, (0,2,1))
Y_t.shape

(2, 1276, 965)

The results of the sklearn_unflatten when transposed, the results of the raster_unstack and the ds_median_array are equal. 

In [37]:
# Test if the results of the transposed sklearn_unflatten and the ds_median_array dataArray are equal. 
np.unique(ds_median_array.values == Y_t) 

array([ True])

In [38]:
# Test if the results of the raster_unstack and the ds_median_array dataArray are equal. 
np.unique(ds_median_array.values == Y1.values) 

array([ True])

In [39]:
# Test if the results of raster_unstack and the results of the transposed sklearn_unflatten are equal.
np.unique(Y_t == Y1.values) 

array([ True])

Time the sklearn_unflatten and the raster_unstack functions. 

In [40]:
%time Y = sklearn_unflatten(X, ds_median_array)

CPU times: user 235 ms, sys: 92.3 ms, total: 327 ms
Wall time: 326 ms


In [41]:
%time Y1 = raster_unstack(X1, ds_median_array)

CPU times: user 16.6 ms, sys: 43 µs, total: 16.6 ms
Wall time: 15.8 ms
